In [287]:
import os
import re
import pandas
import time
import threading
import configparser
import numpy
from urllib import parse

import smtplib
from email.message import EmailMessage
from email.encoders import encode_base64
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from mimetypes import guess_type, read_mime_types

In [357]:
BASE_DIR = os.getcwd()

DOWNLOADS_PATH = os.path.join(os.environ.get('HOMEPATH'), 'Downloads')

FILE_PATH = os.path.join(DOWNLOADS_PATH, 'emails.csv')

CONFIG_PATH = os.path.join(BASE_DIR, 'credentials.ini')

if not os.path.exists(FILE_PATH):
    raise FileExistsError('Could not find the following file.')

In [358]:
data = pandas.read_csv(FILE_PATH)

In [361]:

df = pandas.DataFrame(data=data, columns=['Name', 'Surname', 'Email'])
df.head()

,Name,Surname,Email
0,Thomas,Sauzedde,sauzeddethomas@gmail.com
1,Constantin,Pellissier,cp@tripleacapital.com
2,Philippe,GuignŽ,pg@virtualregatta.com
3,Marc,Charrondire,mcharrondiere@wanadoo.fr
4,Edouard,Level,edlevel@gmail.com


In [362]:
# # Optionally: Transform column names
df = df.rename(columns={'Name': 'name', 'Surname': 'surname', 'Email': 'email'})
df.head()

,name,surname,email
0,Thomas,Sauzedde,sauzeddethomas@gmail.com
1,Constantin,Pellissier,cp@tripleacapital.com
2,Philippe,GuignŽ,pg@virtualregatta.com
3,Marc,Charrondire,mcharrondiere@wanadoo.fr
4,Edouard,Level,edlevel@gmail.com


In [363]:
# Normalize each emails so that we do have difficulties
# when attempting to the send to them
def normalize(email):
    has_matched = re.search(r'^.*(?=\@).*$', str(email))
    if has_matched:
        return has_matched.group(0).lower().strip()
    return None

df['email'] = df['email'].apply(normalize)
df.head()

,name,surname,email
0,Thomas,Sauzedde,sauzeddethomas@gmail.com
1,Constantin,Pellissier,cp@tripleacapital.com
2,Philippe,GuignŽ,pg@virtualregatta.com
3,Marc,Charrondire,mcharrondiere@wanadoo.fr
4,Edouard,Level,edlevel@gmail.com


In [364]:
# Detect and drop empty email fields
df = df[df['email'].isna() == False]
df.shape

(703, 3)

In [365]:
# Convert into a numpy array that
# can be used by the email class
emails_array = numpy.array(df['email'])
emails_array[:3]

array(['sauzeddethomas@gmail.com', 'cp@tripleacapital.com',
       'pg@virtualregatta.com'], dtype=object)

In [366]:
class OfferEmail:
    config = configparser.ConfigParser()

    def __init__(self, setting='google'):
        self.config.read(CONFIG_PATH)

        host = self.config[setting]['host']
        port = self.config['default']['port']
        
        user = self.config[setting]['user']
        password = self.config[setting]['password']

        try:
            connection = smtplib.SMTP(host, port)
        except smtplib.SMTPConnectError:
            raise
        else:
            print('1.', 'Connecting to "%s"' % host)
            connection.ehlo()
            connection.starttls()
            connection.ehlo()
            try:
                connection.login(user, password)
            except smtplib.SMTPAuthenticationError:
                raise
            else:
                print('2.', 'Login successful @ %s' % user)
                self.connection = connection

    def construct_inner_links(self):
        website = ''
        product_link
        return website, product_link

    def send(self, sender, receiver, subject, **kwargs):
        message = MIMEMultipart('alternative')
        message['From'] = sender
        message['To'] = receiver
        message['BCC'] = kwargs['bcc'] if 'bcc' in kwargs else None
        message['Subject'] = subject

        website = 'https://nawoka.fr/'
        website = self.config['default']['website']
        utm_params = {
            'utm_source': 'link',
            'utm_medium': 'email'
        }
        params = parse.urlencode(utm_params)
        website_link = f'{website}?{params}'

        product_link = parse.urljoin(website, 'shop/products/femme/tops/23/crop-top-simple-bretelles')
        product_link = f'{product_link}?{params}&utm_campaign=Special+Offer'

        plain_text = """
        Bonjour,

        Découvrez notre promotion spéciale pour l'été sur votre toute nouvelle plateforme e-commerce de mode Nawoka. 
        En suivant ce lien, vous pourrez bénéficier de -30% sur ce {product_link} crop top unique</a> en série limitée 💓💓

        Pour toutes questions ou informations complémentaires, n'hésitez pas à nous contacter en répondant directement à ce mail.

        Bon shopping sur notre site ! 😊

        ---
        NAWOKA.FR
        {website_link}
        """

        html = f"""
        <html>
            <body>
                <p>
                    Bonjour,
                </p>
                <p>
                    Découvrez notre promotion spéciale pour l'été sur votre toute nouvelle plateforme e-commerce de mode Nawoka. 
                    En suivant ce lien, vous pourrez bénéficier de <strong style='color: blue;'>-30% sur ce 
                    <a href='{product_link}'>crop top unique</a> en série limitée</strong> 💓💓
                </p>
                <p>
                    Pour toutes questions ou informations complémentaires, n'hésitez pas à nous contacter en répondant directement à ce mail.
                </p>
                <p>
                    Bon shopping sur notre site ! 😊
                </p>
                <p>---</p>
                <p style='font-weight: bold;>
                    NAWOKA.FR
                </p>
                <p>
                    <a href='{website_link}'>nawoka.fr</a>
                </p>
            </body>
        </html>
        """

        text = MIMEText(plain_text, 'plain')
        html = MIMEText(html, 'html')

        message.attach(text)
        message.attach(html)

        print('...', 'Sending emails')
        self.connection.sendmail(sender, receiver, message.as_string())
        print('3.', 'Closing connection')
        self.connection.close()

In [367]:
def send_emails_wrapper():
    emailer = OfferEmail(setting='google')

    def construct_emails(debug=False):
        if not debug:
            emails = list(emails_array)
            main_email = emails.pop(0)
            return main_email, ','.join(emails_array)
        return 'hugo_tachini@yahoo.com'

    emails = construct_emails()
    subject = "-30% sur ce crop top tendance pour l'été 💓💓"
    emailer.send('john.pendenque@nawoka.fr', emails[0], subject, bcc=emails[1])

send_emails_wrapper()

1. Connecting to "smtp.gmail.com"
2. Login successful @ contact.nawoka@gmail.com
... Sending emails
3. Closing connection


In [368]:
def write_emails_to_new_file():
    df.to_csv(os.path.join(BASE_DIR, 'emails.csv'))

write_emails_to_new_file()